In [1]:
import os
from langchain_cohere import ChatCohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_neo4j import Neo4jChatMessageHistory, Neo4jGraph
from uuid import uuid4
from dotenv import load_dotenv

load_dotenv()

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    )
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while (q := input("> ")) != "exit":

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: 348b3ed6-1e94-4dc0-b073-d6f82cb6bf0a


c:\Users\jarmo\anaconda3\envs\learn_langchain-env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Hello there! How can I assist you today? I'm here to help with any questions or topics you'd like to discuss.
That's a great idea! Movies are a fantastic way to spend some leisure time. With so many options available, it can be a bit overwhelming to choose. Here's a more personalized list based on the genres you might enjoy:

- **Action-Packed Thrills:**
   - If you're a fan of intense action, consider watching the 'John Wick' series. It follows the journey of a legendary hitman and offers some of the most well-choreographed fight scenes.
   - 'Inception' is another mind-bending action-adventure starring Leonardo DiCaprio. It explores the concept of entering and manipulating dreams.

- **Heartwarming Dramas:**
   - 'Forrest Gump' is a classic drama that will take you on an emotional journey through history. It's a story of love, friendship, and destiny.
   - 'The Pursuit of Happyness' is a powerful and inspiring true story starring Will Smith, showcasing the power of determination.

- 

In [ ]:
import os
from langchain_cohere import ChatCohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_neo4j import Neo4jChatMessageHistory, Neo4jGraph
from uuid import uuid4

SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

llm = ChatCohere(
    cohere_api_key=os.getenv("COHERE_API_KEY")
)

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

youtube = YouTubeSearchTool()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while (q := input("> ")) != "exit":
    
    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

Session ID: 7e447256-480b-423b-b120-bab23a4c7e73


c:\Users\jarmo\anaconda3\envs\learn_langchain-env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Here are two links to the movie trailer for "The Matrix":

https://www.youtube.com/watch?v=vKQi3bBA1y8&pp=ygUTVGhlIE1hdHJpeCB0cmFpbGVyTw%3D%3D
https://www.youtube.com/watch?v=9ix7TUGVYIo&pp=ygUTVGhlIE1hdHJpeCB0cmFpbGVyTw%3D%3D

I hope this is what you were looking for!
I'm sorry, I don't understand your request. Could you please clarify what you are looking for?
